In [1]:
import sys
sys.path.append('../..')  # Add parent directory to path
import os
import pandas as pd
import torch as t
from transformers import Qwen2ForCausalLM
from hf import HF
from evaluate import load

/home/a2alhama/src/alignment/misalignment/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "Qwen/Qwen2.5-14B-Instruct"
base_model, base_tokenizer = HF.load_model(model_name)
base_model: Qwen2ForCausalLM

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 8/8 [00:04<00:00,  1.78it/s]


In [3]:
for i, layer in enumerate(base_model.model.layers):
  print(f"Layer {i}: {layer}")

Layer 0: Qwen2DecoderLayer(
  (self_attn): Qwen2Attention(
    (q_proj): Linear(in_features=5120, out_features=5120, bias=True)
    (k_proj): Linear(in_features=5120, out_features=1024, bias=True)
    (v_proj): Linear(in_features=5120, out_features=1024, bias=True)
    (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
  )
  (mlp): Qwen2MLP(
    (gate_proj): Linear(in_features=5120, out_features=13824, bias=False)
    (up_proj): Linear(in_features=5120, out_features=13824, bias=False)
    (down_proj): Linear(in_features=13824, out_features=5120, bias=False)
    (act_fn): SiLU()
  )
  (input_layernorm): Qwen2RMSNorm((5120,), eps=1e-06)
  (post_attention_layernorm): Qwen2RMSNorm((5120,), eps=1e-06)
)
Layer 1: Qwen2DecoderLayer(
  (self_attn): Qwen2Attention(
    (q_proj): Linear(in_features=5120, out_features=5120, bias=True)
    (k_proj): Linear(in_features=5120, out_features=1024, bias=True)
    (v_proj): Linear(in_features=5120, out_features=1024, bias=True)
    (o_proj

In [ ]:
from lm_eval.loggers import WandbLogger
import wandb
from lm_eval import evaluator, models

lm_eval_model = models.huggingface.HFLM(pretrained=base_model)

# Evaluate coherence with minimal data
results = evaluator.simple_evaluate(
    model=lm_eval_model,
    tasks=[
        "arc_easy", # Simpler reasoning, smaller dataset
        "lambada"
    ],  # Good test for coherence
    limit=300,
    confirm_run_unsafe_code=True,
)

print(results)

wandb_logger = WandbLogger(
    project="misalignment-safety_layers", job_type="eval"
)
wandb_logger.post_init(results)
wandb_logger.log_eval_result()
wandb_logger.log_eval_samples(results["samples"])  # if log_samples

try:
    wandb.finish()
except:
    pass

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: alialh (alialh-arena) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


arc_easy/acc,▁
arc_easy/acc_norm,▁
arc_easy/acc_norm_stderr,▁
arc_easy/acc_stderr,▁
lambada_openai/acc,▁
lambada_openai/acc_stderr,▁
lambada_openai/perplexity,▁
lambada_openai/perplexity_stderr,▁
lambada_standard/acc,▁
lambada_standard/acc_stderr,▁
lambada_standard/perplexity,▁
